In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [2]:
terms = {
        'BonusMalus': {'type': 'ns'}, 
        'VehAge': {'type': 'ns'},
        'DrivAge': {'type': 'ns'}, 
        'VehPower': {'type': 'ns'}, 
        'Area': {'type': 'target_encoding'},
        'Region': {'type': 'target_encoding'},
        'VehGas': {'type': 'target_encoding'},
        'VehBrand': {'type': 'target_encoding'}
    }

interactions=[{
        "VehPower": {"type": "linear"},
        "Region": {"type": "categorical"},
        "target_encoding": True,
        "include_main": False
    }]

result = rs.glm_dict(
    response='ClaimCount',
    terms=terms,
    interactions=interactions,
    data=data,
    family='poisson',
    offset ='Exposure'
).fit()

In [3]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ ns(BonusMalus, df=4) + ns(VehAge, df=4) + ns(DrivAge, df=4) + ns(VehPower, df=4) + TE(Area) + TE(Region) + TE(VehGas) + TE(VehBrand)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 42, 'df_resid': 406397, 'converged': True, 'iterations': 7}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21323.37, loss=0.309509, deviance=125796.67, log_likelihood=-83672.16, aic=167428.32, gini=0.2846, auc=0.6423, ae_ratio=1.004, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1926.83, actual=0.38872, predicted=0.075423, ae_ratio=5.1539), DecileMetrics(decile=2, n=40643, exposure=4242.15, actual=0.224886, predicted=0.088946, ae_ratio=2.5284), DecileMetrics(decile=3, n=40643, exposure=8292.17, actual=0.163287, predicted=0.08462, ae_ratio=1.9297), DecileMetrics(decile=4, n=40643, exposure=14055.29, actual=0.119528, predic

In [4]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        678012
Link Function:       (default)       Df Residuals:            677970
Method:              IRLS + Smooth   Df Model:                    41
Scale:               1.0000          Alpha (λ):               0.0000
L1 Ratio:            0.00            Iterations:                   7
Non-zero coefs:      41             

Log-Likelihood:         -139170.1585 Deviance:                209340.1708
AIC:                     278424.3170 Null Deviance:           220244.0296
BIC:                     278904.2476 Pearson chi2:             1523409.19
Converged:           True           

------------------------------------------------------------------------------
Variable                  Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept              -2.7307 

In [5]:
result.predict(holdout_data)

array([0.05211679, 0.04871169, 0.04870507, ..., 0.00555171, 0.01725124,
       0.07213412], shape=(135957,))

In [6]:
# Model spec before interaction
terms = {
    "BonusMalus": {"type": "linear"},
    "VehAge": {"type": "ns", "df": 5},
    "DrivAge": {"type": "linear"},
    "Region": {"type": "target_encoding"},
    "VehBrand": {"type": "target_encoding"},
    "Area": {"type": "target_encoding"},
}
# Interaction to add (DrivAge x BonusMalus)
interactions = [
    {
        "DrivAge": {"type": "linear"},
        "Region": {"type": "target_encoding"},
        "include_main": False,
    }
]
# Fit with interaction
result = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit()


In [7]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406428
Method:              IRLS            Df Model:                    10
Scale:               1.0000          Iterations:                   6

Log-Likelihood:          -84125.3896 Deviance:                126703.1296
AIC:                     168272.7793 Null Deviance:           132328.3090
BIC:                     168392.8464 Pearson chi2:              935662.92
Converged:           True           

------------------------------------------------------------------------------
Variable                 Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept             -4.3831     0.1635  -26.801  <0.0001   [ -4.7037,  -4.0626]  ***
BonusMalus             0.0219     0.0004   54.644  

In [8]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [9]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21351.71, loss=0.311694, deviance=126684.47, log_likelihood=-84116.06, aic=168254.12, gini=0.2596, auc=0.6298, ae_ratio=1.0026, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1902.34, actual=0.41265, predicted=0.077795, ae_ratio=5.3044), DecileMetrics(decile=2, n=40643, exposure=4171.8, actual=0.236828, predicted=0.091234, ae_ratio=2.5958), DecileMetrics(decile=3, n=40643, exposure=8098.3, actual=0.165343, predicted=0.087388, ae_ratio=1.8921), DecileMetrics(decile=4, n=40643, exposure=13729.54, actual=0.127535, predicted=0.082841, ae_ratio=1.5395), DecileMetrics(decile=5

In [10]:
predictions = result.predict(holdout_data)

In [11]:
model_bytes = result.to_bytes()
with open("model.bin", "wb") as f:
    f.write(model_bytes)

In [12]:
with open("model.bin", "rb") as f:
    loaded = rs.GLMModel.from_bytes(f.read())

In [13]:
predictions2 = loaded.predict(holdout_data)

In [14]:
import numpy as np
print(np.array_equal(predictions2, predictions))

True


In [15]:
#writes analysis/diagnostics
loaded.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21351.71, loss=0.311694, deviance=126684.47, log_likelihood=-84116.06, aic=168254.12, gini=0.2596, auc=0.6298, ae_ratio=1.0026, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1902.34, actual=0.41265, predicted=0.077795, ae_ratio=5.3044), DecileMetrics(decile=2, n=40643, exposure=4171.8, actual=0.236828, predicted=0.091234, ae_ratio=2.5958), DecileMetrics(decile=3, n=40643, exposure=8098.3, actual=0.165343, predicted=0.087388, ae_ratio=1.8921), DecileMetrics(decile=4, n=40643, exposure=13729.54, actual=0.127535, predicted=0.082841, ae_ratio=1.5395), DecileMetrics(decile=5

In [16]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"}
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net"
)

In [17]:
#writes analysis/diagnostics
result2.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ TE(Region) + BonusMalus + I(BonusMalus ** 2)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 4, 'df_resid': 406435, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21448.32, loss=0.318666, deviance=129518.12, log_likelihood=-85532.89, aic=171073.77, gini=0.1841, auc=0.592, ae_ratio=0.9981, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1694.73, actual=0.640811, predicted=0.089131, ae_ratio=7.1895), DecileMetrics(decile=2, n=40643, exposure=3748.89, actual=0.287552, predicted=0.099283, ae_ratio=2.8963), DecileMetrics(decile=3, n=40643, exposure=7518.99, actual=0.18287, predicted=0.0952, ae_ratio=1.9209), DecileMetrics(decile=4, n=40643, exposure=12520.26, actual=0.138815, predicted=0.095159, ae_ratio=1.4588), DecileMetrics(decile=5, n=40643, exposure=18500.66, actual

In [18]:
result3 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "BonusMalus": {"type": "ns"},
        "VehAge": {"type": "ns"},
        "DrivAge": {"type": "linear"},
        "VehBrand": {"type": "target_encoding"}
    },
    interactions= [
        {
            "VehAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        },
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ],
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net",
    include_unregularized=False
)

In [19]:
import rustystats as rs

# Model spec that caused the issue when elastic_net was applied
terms = {
    "BonusMalus": {'type' : "linear"},
    "VehAge": {'type' : "ns"}, 
    "DrivAge": {'type' : "ns"},
    "Area": {'type' : "target_encoding"},
}

interactions= [
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ]

# Build the model
glm = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,  # freMTPL2freq dataset
    family="poisson",
    offset="Exposure",  # log-linked
)

# This is where the issue occurs - regularized fit + diagnostics
result = glm.fit()

# Check if diagnostics returns None for deviance
diag = result.diagnostics(train_data=train_data, test_data=holdout_data)
print(diag.to_dict())  # Check train_test.test.deviance

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 22 is different from 30)